# Interface

"Runnable" 프로토콜은 사용자 정의 체인을 가능한 한 쉽게 만들 수 있도록 설계되었습니다. 대부분의 구성 요소에 대해 구현되어 있으며, 이는 표준 인터페이스를 통해 사용자 정의 체인을 정의하고 표준 방식으로 호출할 수 있게 합니다. 

이 표준 인터페이스는 다음을 포함합니다:

1. stream: 응답의 청크를 스트리밍 방식으로 반환합니다.
2. invoke: 입력에 대해 체인을 호출합니다.
3. batch: 입력 목록에 대해 체인을 호출합니다.

이러한 메소드에는 해당하는 비동기 메소드도 있어, 다양한 실행 환경과 요구 사항에 따라 유연하게 
대응할 수 있습니다:

1. astream: 응답의 청크를 비동기로 스트리밍 방식으로 반환합니다.
2. ainvoke: 입력에 대해 체인을 비동기로 호출합니다.
3. abatch: 입력 목록에 대해 체인을 비동기로 호출합니다.
3. astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍 방식으로 반환합니다. 이를 통해 개발자는 체인의 실행 과정을 더 잘 이해하고 디버깅할 수 있습니다.
5. astream_events: 체인에서 발생하는 이벤트를 실시간으로 스트리밍 방식으로 반환합니다. 이는 langchain-core 0.1.14에서 도입된 베타 기능으로, 체인의 동작을 더 세밀하게 추적할 수 있게 합니다.

이 인터페이스는 LCEL을 사용하는 개발자가 더 복잡하고 다기능적인 애플리케이션을 손쉽게 구축할 수 있도록 지원합니다. 비동기 실행 지원은 특히 현대적인 웹 애플리케이션 및 데이터 처리 워크플로우에서 중요한 역할을 하며, LCEL 체인을 사용하여 고성능 및 대응성 높은 시스템을 설계할 수 있습니다.

각 구성 요소의 입력 유형과 출력 유형은 구성 요소에 따라 다릅니다:
| Component    | Input Type                                | Output Type               |
|--------------|-------------------------------------------|---------------------------|
| Prompt       | 사전(Dictionary)                           | PromptValue               |
| ChatModel    | 단일 문자열, 채팅 메시지 목록 또는 PromptValue    | ChatMessage   |
| LLM          | 단일 문자열, 채팅 메시지 목록 또는 PromptValue    | 문자열(String)        |
| OutputParser | LLM 또는 ChatModel의 출력        | 파서에 따라 다름      |
| Retriever    | 단일 문자열                             | 문서 목록(List of Documents)         |
| Tool         | 단일 문자열 또는 도구에 따라 사전 | 도구에 따라 다름|

모든 Runnable은 입력과 출력 스키마를 노출하여 Runnable의 구조로부터 자동 생성된 입력 및 출력 Pydantic 모델을 검사할 수 있습니다:

`input_schema`: Runnable의 구조로부터 자동 생성된 입력 Pydantic 모델

`output_schema`: Runnable의 구조로부터 자동 생성된 출력 Pydantic 모델
이 메소드들을 살펴보겠습니다. 이를 위해, 매우 간단한 PromptTemplate + ChatModel 체인을 생성해 보겠습니다.

In [1]:
%pip install --upgrade --quiet langchain-core langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# langchain_core의 prompts 모듈에서 ChatPromptTemplate 클래스를 가져옵니다.
from langchain_core.prompts import ChatPromptTemplate
# langchain_openai 모듈에서 ChatOpenAI 클래스를 가져옵니다.
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델의 인스턴스를 생성합니다. 이 모델은 OpenAI의 GPT 모델을 사용하여 텍스트를 생성합니다.
model = ChatOpenAI()
# "tell me a joke about {topic}" 형태의 프롬프트 템플릿을 생성합니다. {topic}은 사용자로부터 입력받은 주제로 대체됩니다.
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
# 프롬프트와 모델을 연결하여 체인을 생성합니다. 이 체인은 사용자로부터 주제를 입력받아 해당 주제에 관한 농담을 생성하는 데 사용됩니다.
chain = prompt | model

In [ ]:
model.input_schema.schema()


model.input_schema.schema() 호출 결과는 ChatOpenAI 모델이 받아들일 수 있는 입력의 스키마를 보여줍니다. 이 JSON 스키마는 모델이 처리할 수 있는 다양한 유형의 입력을 설명합니다. 다음은 주요 부분의 설명입니다:

In [ ]:
{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'human',
     'enum': ['human'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'ChatMessage': {'title': 'ChatMessage',
   'description': 'A Message that can be assigned an arbitrary speaker (i.e. role).',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'chat',
     'enum': ['chat'],
     'type': 'string'},
    'role': {'title': 'Role', 'type': 'string'}},
   'required': ['content', 'role']},
  'SystemMessage': {'title': 'SystemMessage',
   'description': 'A Message for priming AI behavior, usually passed in as the first of a sequence\nof input messages.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'system',
     'enum': ['system'],
     'type': 'string'}},
   'required': ['content']},
  'FunctionMessage': {'title': 'FunctionMessage',
   'description': 'A Message for passing the result of executing a function back to a model.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'function',
     'enum': ['function'],
     'type': 'string'},
    'name': {'title': 'Name', 'type': 'string'}},
   'required': ['content', 'name']},
  'ToolMessage': {'title': 'ToolMessage',
   'description': 'A Message for passing the result of executing a tool back to a model.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'tool',
     'enum': ['tool'],
     'type': 'string'},
    'tool_call_id': {'title': 'Tool Call Id', 'type': 'string'}},
   'required': ['content', 'tool_call_id']},
  'ChatPromptValueConcrete': {'title': 'ChatPromptValueConcrete',
   'description': 'Chat prompt value which explicitly lists out the message types it accepts.\nFor use in external schemas.',
   'type': 'object',
   'properties': {'messages': {'title': 'Messages',
     'type': 'array',
     'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
       {'$ref': '#/definitions/HumanMessage'},
       {'$ref': '#/definitions/ChatMessage'},
       {'$ref': '#/definitions/SystemMessage'},
       {'$ref': '#/definitions/FunctionMessage'},
       {'$ref': '#/definitions/ToolMessage'}]}},
    'type': {'title': 'Type',
     'default': 'ChatPromptValueConcrete',
     'enum': ['ChatPromptValueConcrete'],
     'type': 'string'}},
   'required': ['messages']}}}

'anyOf': 이 모델은 여러 유형의 입력을 받을 수 있습니다. 단일 문자열, StringPromptValue, ChatPromptValueConcrete, 또는 여러 메시지 유형(AIMessage, HumanMessage, ChatMessage, SystemMessage, FunctionMessage, ToolMessage)의 배열을 입력으로 받을 수 있습니다.

'StringPromptValue'와 다른 메시지 유형들: 이러한 정의는 모델이 처리할 수 있는 특정 메시지 유형을 상세하게 설명합니다. 예를 들어, AIMessage는 AI로부터 온 메시지를, HumanMessage는 인간 사용자로부터 온 메시지를 나타냅니다. 각 메시지 유형은 content 속성을 포함하며, 추가적인 속성을 가질 수도 있습니다.

'ChatPromptValueConcrete': 이 정의는 채팅 프롬프트 값을 나타내며, messages 배열 안에 여러 메시지 유형을 포함할 수 있습니다. 이는 대화식 입력이나 대화 컨텍스트를 모델에 전달할 때 사용됩니다.

이 스키마를 통해 ChatOpenAI 모델에 전달할 수 있는 입력의 다양성과 유연성을 확인할 수 있습니다. 개발자는 이 정보를 바탕으로 모델에 적합한 입력 데이터를 준비하고, 모델이 처리할 수 있는 입력 유형에 대해 더 잘 이해할 수 있습니다. 이는 특히 다양한 대화 시나리오나 복잡한 대화 컨텍스트를 다루는 애플리케이션을 개발할 때 유용합니다.
